In [ ]:
import utils
import stack
import volume
import sandbox
import nibabel as nib
import torch as t
import os
import numpy as np
import open3d as o3d
import pytorch3d as p3

$\textbf{read nifti format and return image, affine and zooms as torch tensor, show slices (not in world coordinates)}$

In [ ]:
folder = "data"
filename = "iFIND00472_iFIND2_201901111210_PHILIPSJ2LF9B9_101_PIH1HR_Survey_32SENSE.s3.nii.gz"
t_image, t_affine, zooms = utils.nii_to_torch(folder, filename)
utils.show_stack(t_image)

$\textbf{Create stack object from subpart of scan}$

In [ ]:
t_image_red = t_image[200:400,200:400,:]
utils.show_stack(t_image_red)

In [ ]:
beta = 0.01
first_stack = stack.stack(t_image_red,t_affine, beta)

$\textbf{define desired geometry and the number of voxels according to resolution}$

In [ ]:
resolution = 0.5
geometry = first_stack.corners()
n_voxels = t.ceil(t.multiply(t.tensor(t_image_red.shape),resolution))
n_voxels[n_voxels < t.min(t.tensor(t_image_red.shape))] = t.min(t.tensor(t_image_red.shape)).item()
n_voxels = n_voxels.int()

$\textbf{create target volume and run registration}$

In [ ]:
target = volume.volume()
target.from_stack(geometry,n_voxels)
target.reconstruct_stack(first_stack)

$\textbf{save the result as nifti and save to folder}$

In [ ]:
nft_img = utils.torch_to_nii(target.X, target.affine)
folder = 'test_reconstruction'
filename = 's1_smaller_jupygter'
utils.save_target(target,folder, filename)
utils.save_nifti(nft_img,folder, filename)

$\textbf{Sampling with 45° around z-Axis of the slices}$

In [ ]:
filename = "s1_cropped.nii"
folder = "data"
t_image, t_affine, zooms = utils.nii_to_torch(folder, filename)

In [ ]:
rotations = t.tensor([0,0,np.pi/4])
I_x, I_y, I_z = 120, 110, 6
I = t.zeros(I_x, I_y, I_z)
t_affine = t.eye(4)
t_affine = utils.create_T(rotations, t.zeros(3))
sampling_stack = stack.stack(I,t_affine)

In [ ]:
folder = 'test_reconstruction'
filename = 's1_cropped_complete'
target_loaded = utils.open_target(folder, filename)

In [ ]:
sampling_stack.sample_from_volume(target_loaded)
#sampling_2d(sampling_stack, target_loaded)
utils.show_stack(sampling_stack.I)

$\textbf{Sampling with 90° around x-Axis of the slices}$

In [ ]:
rotations = t.tensor([np.pi/2,0,0])
I_x, I_y, I_z = 80, 90, 6
sandbox.basic_2d_sampling(rotations, I_x, I_y, I_z)